In [5]:
import random
import dspy
import ollama
import chromadb

# Initialize a random collection name
j = random.randint(0, 29999)

# Create a ChromaDB client
client = chromadb.HttpClient(host='localhost', port=8000)
collection = client.create_collection(name=f"docs{j}")

In [6]:
import ollama
from chromadb.api.types import (
    Documents,
    EmbeddingFunction,
    Embeddings
)

class OllamaEmbeddingFunction(EmbeddingFunction[Documents]):
    def __init__(
            self,
            model_name: str = "mxbai-embed-large"
    ):
        """Initialize the embedding function."""
        self.model_name = model_name

    def __call__(self, input: Documents) -> Embeddings:
        """Embed the input documents."""
        return self._embed(input)

    def _embed(self, documents: Documents) -> Embeddings:
        """Generate embeddings for the input documents using Ollama."""
        embeddings = []
        for doc in documents:
            response = ollama.embeddings(
                model=self.model_name,
                prompt=doc
            )
            embeddings.append(response["embedding"])
        return embeddings

    def _retrieve(self, query: str, n_results: int) -> Documents:
        """Retrieve relevant documents based on a query using Ollama."""
        response = ollama.embeddings(
            model=self.model_name,
            prompt=query
        )
        query_embedding = response["embedding"]

        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results
        )

        return [doc[:n_results] for doc in results['documents']]


In [7]:
embedding_function = OllamaEmbeddingFunction(model_name="mxbai-embed-large")

In [8]:
import requests
from typing import Optional
import dspy  # Assuming dspy is a module you have that includes the Retrieve and Prediction classes
from chromadb.api.types import Documents

class DSPythonicRMClient(dspy.Retrieve):
    def __init__(self, url: str, port: int = None, k: int = 3, embedding_function: OllamaEmbeddingFunction = None):
        super().__init__(k=k)

        self.url = f"{url}:{port}" if port else url
        self.embedding_function = embedding_function

    def forward(self, query_or_queries: str, k: Optional[int] = None) -> dspy.Prediction:
        params = {"query": query_or_queries, "k": k if k else self.k}
        response = requests.get(self.url, params=params)

        response = response.json()["retrieved_passages"]  # List of top k passages
        return dspy.Prediction(
            passages=response
        )

    def embed_query(self, query: str) -> list:

        """Generate embeddings for the query using the OllamaEmbeddingFunction."""

        if self.embedding_function is None:
            raise ValueError("No embedding function provided.")
        

        # Use the embedding function to get the embedding for the query
        return self.embedding_function([query])[0]  # Assuming the embedding function returns a list of embeddings


    def retrieve_with_embedding(self, query: str, k: Optional[int]) -> dspy.Prediction:
        """Retrieve passages based on the embedded query."""
        query_embedding = self.embed_query(query)
        
        # You can now use this embedding to perform retrieval
        # Here, we assume there's a method to query the collection with the embedding
        results = collection.query(query_embeddings=[query_embedding], n_results =k if k else self.k)

        return dspy.Prediction(
            passages=[doc[k] for doc in results['documents']]
        )




In [9]:
import ollama
import chromadb
import random 
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
""" nltk.download('punkt')
nltk.download("punkt_tab") """

story="""

## The Hidden Garden.

In a small town, there was a garden hidden behind a tall wall. The townsfolk often whispered about it, saying it was enchanted. No one had seen the inside, but everyone had their own stories.

Some said it was filled with flowers that could sing, while others believed it was home to a wise old owl that could grant wishes. 

One sunny afternoon, a curious girl named Lily decided to explore the garden. 

Lilly had always been fascinated by the stories and wanted to see if they were true. 

As she approached the wall, she noticed a small, rusty gate slightly ajar. 

With her heart racing, she pushed it open and stepped inside.

The garden was more beautiful than she had imagined. 

Colorful flowers bloomed everywhere, and the air was filled with sweet scents. 

In the center stood an ancient oak tree, its branches stretching wide, offering shade to a small stone bench. 

As she walked closer, she noticed a figure sitting on the bench—a boy about her age, with messy brown hair and bright green eyes.

“Hi, I’m Jack,” he said, smiling. “I come here to think.”

“Hi, I’m Lily,” she replied, feeling a strange connection with him. “I’ve heard so many stories about this place.”

Jack nodded. “Most of them are true. The flowers do sing, but only when they feel happy. And the owl? He’s real too. He watches over the garden.”

Lily’s eyes widened. “Can we see him?”

Jack hesitated. “He only appears when he senses someone who truly believes in magic.”

Lily felt a spark of determination. “I believe! Let’s find him!”

Together, they explored the garden, laughing and sharing stories. They discovered a patch of flowers that began to hum softly when they danced around them. The more they danced, the louder the flowers sang, filling the air with joy.

As they played, they talked about their lives. Lily shared how she often felt lonely at school, while Jack revealed that he had a sick sister who couldn’t leave their home. They both understood what it felt like to wish for something more.

Suddenly, the air grew still, and a soft hoot echoed through the garden. They turned to see a majestic owl perched on a low branch of the oak tree. Its feathers shimmered in the sunlight, and its eyes sparkled with wisdom.

“Welcome, young ones,” the owl said in a deep, soothing voice. “You have shown kindness and belief. What is it that you seek?”

Lily and Jack exchanged glances, unsure of what to say. Finally, Lily spoke up. “I wish for Jack’s sister to get better.”

The owl nodded slowly. “True friendship and selflessness are the greatest forms of magic. I will grant your wish, but remember, kindness must be shared.”

With a flap of its wings, the owl vanished into the air, leaving behind a soft glow. Jack looked at Lily, tears of gratitude in his eyes. “Thank you, Lily. You’ve given me hope.”

As the sun began to set, they promised to meet again in the garden. They had found not only magic but also a friendship that would last a lifetime.



"""
#
documents = sent_tokenize(story)
for i in range(len(documents)):
    documents[i] = documents[i].replace('\n', '')


#print(documents)

In [10]:


# Create an instance of the embedding function
oef = OllamaEmbeddingFunction(model_name="mxbai-embed-large")

col=oef(documents)

questions = [
"What was the main reason Lily decided to explore the hidden garden?",
"How did Jack describe the flowers in the garden?",
"What personal struggles did Lily and Jack share with each other?",
"What did the owl say when it first appeared to Lily and Jack?",
"What wish did Lily make for Jack?",
"How did the flowers react when Lily and Jack danced around them?",
"What did the owl mean by saying 'kindness must be shared'?",
"How did the setting of the garden contribute to the story?",
"What emotions did Jack express after Lily made her wish?",
"How did the relationship between Lily and Jack develop throughout the story?"
]


# Create an instance of DSPythonicRMClient
client = DSPythonicRMClient(url="http://localhost", port=8000, k=2, embedding_function=oef)

# Retrieve passages based on a query
query = questions[1]
results = client(query)

# Print the retrieved passages
for passage in results.passages:
    print(passage) 


KeyError: 'retrieved_passages'